
# Reactiva Perú 2022 — Análisis y Modelado (CRISP-DM)

**Autores:**  
- Luis Lucero  
- Dayvis Quispe  

**Fuente de datos (Reactiva Perú 2022):**  
https://www.mef.gob.pe/contenidos/archivos-descarga/REACTIVA_Lista_beneficiarios_270422.xlsx

> Este notebook adapta la **estructura** del código original en **R (RStudio)** a **Python (Jupyter)**, manteniendo la misma lógica por **fases** y actualizando descripciones para mayor claridad. Se emplea `pandas` para manejo de datos, `matplotlib` para visualización y `scikit-learn` para preparación y modelado (penalización L1 equivalente al **Lasso** en regresión logística).


## 0. Configuración e instalación opcional

In [ ]:

# Si necesitas instalar dependencias en un entorno limpio, descomenta lo siguiente:
# %pip install pandas openpyxl scikit-learn matplotlib

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    confusion_matrix, accuracy_score, classification_report,
    roc_auc_score, roc_curve
)

pd.set_option('display.max_columns', 200)



## 1. Importación y preparación de datos (CRISP-DM: Data Understanding & Preparation)

**Objetivo:** Cargar, estandarizar nombres de columnas, tipificar variables (categóricas/numéricas), y crear la variable objetivo **REPRO** en formato binario.

**Notas de adaptación:**
- Se renombra igual que en R: campos con acentos y espacios → **snake case** sin acentos.
- Se convierte `REPRO` de {"SI","NO"} a {1,0}.
- Se usa `category` en pandas para variables categóricas, análogo a `factor` en R.


In [ ]:

# Ruta del archivo (ajusta si es necesario)
DATA_PATH = "Reactiva_Perú 2022.xlsx"  # O "Reactiva_Perú 2022.xlsx" según el nombre exacto del archivo

# Carga
try:
    df = pd.read_excel(DATA_PATH)
except Exception as e:
    print("⚠️ No se pudo leer el archivo Excel. Asegúrate del nombre/ruta y del motor 'openpyxl' instalado.")
    raise e

# Renombrar columnas para estandarizar
rename_map = {
    "RUC/DNI": "RUC_O_DNI",
    "RAZÓN SOCIAL": "RAZON_SOCIAL",
    "RAZON SOCIAL": "RAZON_SOCIAL",
    "SECTOR ECONÓMICO": "SECTOR_ECONOMICO",
    "SALDO INSOLUTO (S/)": "SALDO_INSOLUTO",
    "COBERTURA DEL SALDO INSOLUTO(S/)": "COBERTURA_SALDO_INSOLUTO",
    "NOMBRE DE ENTIDAD OTORGANTE DEL CRÉDITO": "ENTIDAD_OTORGANTE_CREDITO"
}

df = df.rename(columns={k: v for k, v in rename_map.items() if k in df.columns})

# Conversiones de tipo
for col in ["RAZON_SOCIAL", "RUC_O_DNI", "SECTOR_ECONOMICO", "ENTIDAD_OTORGANTE_CREDITO", "DEPARTAMENTO"]:
    if col in df.columns:
        df[col] = df[col].astype("category")

# Variable objetivo REPRO a binario (1 = SI, 0 = NO)
if "REPRO" in df.columns:
    df["REPRO"] = df["REPRO"].astype(str).str.upper().str.strip().map({"SI": 1, "NO": 0})
else:
    raise ValueError("No se encontró la columna 'REPRO' en el dataset.")

# Información general
display(df.head())
print("\nEstructura (info):\n")
print(df.info())

# Correlación de Spearman para numéricas relevantes (ej. SALDO y COBERTURA)
num_cols = [c for c in ["SALDO_INSOLUTO", "COBERTURA_SALDO_INSOLUTO"] if c in df.columns]
if len(num_cols) == 2:
    corr_spear = df[num_cols].corr(method="spearman")
    print("\nCorrelación de Spearman entre SALDO_INSOLUTO y COBERTURA_SALDO_INSOLUTO:")
    print(corr_spear)
else:
    print("\nNo se encontraron ambas columnas numéricas esperadas para correlación.")



## 2. Análisis descriptivo (CRISP-DM: Data Understanding)

Se replican y mejoran las visualizaciones solicitadas:
1. **Distribución de REPRO** (gráfico circular).  
2. **Frecuencia y porcentaje por DEPARTAMENTO** (barras horizontales).  
3. **Frecuencia y porcentaje por SECTOR_ECONOMICO** (barras horizontales).  
4. **Frecuencia por SECTOR_ECONOMICO segmentada por REPRO** (barras agrupadas).

> Usamos **matplotlib** y evitamos estilos/colores específicos para mantener neutralidad y compatibilidad.


In [ ]:

# 2.1 Pie chart de REPRO
if "REPRO" in df.columns:
    repro_counts = df["REPRO"].value_counts().sort_index()  # 0,1
    labels = ["NO", "SI"]
    sizes = [repro_counts.get(0, 0), repro_counts.get(1, 0)]
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.pie(sizes, labels=labels, autopct="%1.1f%%", startangle=90)
    ax.set_title("Porcentaje de empresas con REPRO")
    ax.axis("equal")
    plt.show()
else:
    print("No se puede graficar REPRO (columna ausente).")

# 2.2 Barras por DEPARTAMENTO (frecuencia y porcentaje)
if "DEPARTAMENTO" in df.columns:
    dept_counts = df["DEPARTAMENTO"].value_counts()
    fig, ax = plt.subplots(figsize=(8, max(4, len(dept_counts) * 0.3)))
    dept_counts.sort_values().plot(kind="barh", ax=ax)
    ax.set_title("Frecuencia de empresas según región (DEPARTAMENTO)")
    ax.set_xlabel("Frecuencia")
    ax.set_ylabel("Región (Departamento)")
    # Etiquetas de conteo
    for i, v in enumerate(dept_counts.sort_values().values):
        ax.text(v + max(dept_counts) * 0.01, i, str(v), va="center")
    plt.tight_layout()
    plt.show()

    # Porcentaje por departamento
    dept_pct = (dept_counts / dept_counts.sum() * 100).sort_values()
    fig, ax = plt.subplots(figsize=(8, max(4, len(dept_pct) * 0.3)))
    dept_pct.plot(kind="barh", ax=ax)
    ax.set_title("Porcentaje de empresas según región (DEPARTAMENTO)")
    ax.set_xlabel("Porcentaje (%)")
    ax.set_ylabel("Región (Departamento)")
    for i, v in enumerate(dept_pct.values):
        ax.text(v + max(dept_pct) * 0.01, i, f"{v:.1f}%", va="center")
    plt.tight_layout()
    plt.show()
else:
    print("No se puede graficar por DEPARTAMENTO (columna ausente).")

# 2.3 Barras por SECTOR_ECONOMICO (frecuencia y porcentaje)
if "SECTOR_ECONOMICO" in df.columns:
    sec_counts = df["SECTOR_ECONOMICO"].value_counts()
    fig, ax = plt.subplots(figsize=(8, max(4, len(sec_counts) * 0.28)))
    sec_counts.sort_values().plot(kind="barh", ax=ax)
    ax.set_title("Frecuencia de empresas según sector económico")
    ax.set_xlabel("Frecuencia")
    ax.set_ylabel("Sector Económico")
    for i, v in enumerate(sec_counts.sort_values().values):
        ax.text(v + max(sec_counts) * 0.01, i, str(v), va="center")
    plt.tight_layout()
    plt.show()

    sec_pct = (sec_counts / sec_counts.sum() * 100).sort_values()
    fig, ax = plt.subplots(figsize=(8, max(4, len(sec_pct) * 0.28)))
    sec_pct.plot(kind="barh", ax=ax)
    ax.set_title("Porcentaje de empresas según sector económico")
    ax.set_xlabel("Porcentaje (%)")
    ax.set_ylabel("Sector Económico")
    for i, v in enumerate(sec_pct.values):
        ax.text(v + max(sec_pct) * 0.01, i, f"{v:.1f}%", va="center")
    plt.tight_layout()
    plt.show()
else:
    print("No se puede graficar por SECTOR_ECONOMICO (columna ausente).")

# 2.4 Barras por SECTOR_ECONOMICO segmentadas por REPRO (agrupadas)
if "SECTOR_ECONOMICO" in df.columns and "REPRO" in df.columns:
    ct = pd.crosstab(df["SECTOR_ECONOMICO"], df["REPRO"]).sort_values(by=[0,1], ascending=True)
    # Preparar gráfico agrupado manualmente
    indices = np.arange(ct.shape[0])
    width = 0.4
    fig, ax = plt.subplots(figsize=(10, max(4, ct.shape[0] * 0.28)))
    ax.barh(indices - width/2, ct[0].values if 0 in ct.columns else np.zeros_like(indices), height=width, label="REPRO = NO")
    ax.barh(indices + width/2, ct[1].values if 1 in ct.columns else np.zeros_like(indices), height=width, label="REPRO = SI")
    ax.set_yticks(indices)
    ax.set_yticklabels(ct.index.tolist())
    ax.set_xlabel("Frecuencia de empresas")
    ax.set_title("Frecuencia por sector económico y REPRO")
    ax.legend(loc="lower right")
    # Etiquetas
    for i, v in enumerate(ct[0].values if 0 in ct.columns else np.zeros_like(indices)):
        ax.text(v + (ct.values.max() * 0.01 if ct.values.size else 0.5), i - width/2, str(v), va="center")
    for i, v in enumerate(ct[1].values if 1 in ct.columns else np.zeros_like(indices)):
        ax.text(v + (ct.values.max() * 0.01 if ct.values.size else 0.5), i + width/2, str(v), va="center")
    plt.tight_layout()
    plt.show()
else:
    print("No se puede graficar barras agrupadas (faltan columnas SECTOR_ECONOMICO/REPRO).")



## 3. Desarrollo del modelo (CRISP-DM: Modeling & Evaluation)

**Modelo:** Regresión Logística con penalización **L1** (equivalente conceptual a **Lasso** para clasificación), usando `solver='saga'` y búsqueda de hiperparámetro `C` con validación cruzada.

**Variables usadas (como en R):**
- **Predictoras:** `SECTOR_ECONOMICO`, `ENTIDAD_OTORGANTE_CREDITO`, `DEPARTAMENTO`, `SALDO_INSOLUTO`  
- **Objetivo:** `REPRO` (1 = SI, 0 = NO)

**Estrategia de partición:** Estratificación por `REPRO` (garantiza balance de clases en train/test).  
> En el script de R se creó una interacción de varias columnas para estratificar; en la práctica, estratificar por la clase objetivo suele ser suficiente y más estable con alto cardinalidad.


In [ ]:

# Selección de columnas
cols_existentes = set(df.columns)
features = [c for c in ["SECTOR_ECONOMICO", "ENTIDAD_OTORGANTE_CREDITO", "DEPARTAMENTO", "SALDO_INSOLUTO"] if c in cols_existentes]
target = "REPRO"
assert target in df.columns, "No se encuentra la columna objetivo 'REPRO'."

X = df[features].copy()
y = df[target].astype(int).values

# Tipos de columnas
cat_cols = [c for c in features if df[c].dtype.name == "category" or df[c].dtype == object]
num_cols = [c for c in features if c not in cat_cols]

# Partición estratificada
sss = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=123)
train_idx, test_idx = next(sss.split(X, y))
X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

# Preprocesamiento
preprocess = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(with_mean=True, with_std=True), num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse=False), cat_cols),
    ]
)

# Pipeline: Prepro + Clasificador (L1 Logistic)
pipe = Pipeline(steps=[
    ("preprocess", preprocess),
    ("clf", LogisticRegression(
        penalty="l1",
        solver="saga",
        max_iter=5000,
        n_jobs=None,  # usar CPUs por defecto
        class_weight=None
    ))
])

# Búsqueda de C (inverso de la regularización)
param_grid = {"clf__C": np.logspace(-3, 2, 12)}
cv = GridSearchCV(pipe, param_grid=param_grid, cv=5, scoring="accuracy", n_jobs=-1, refit=True, verbose=0)
cv.fit(X_train, y_train)

best_model = cv.best_estimator_
print("Mejor parámetro C:", cv.best_params_)

# Evaluación en test
y_proba = best_model.predict_proba(X_test)[:, 1]
y_pred = (y_proba >= 0.5).astype(int)

acc = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred, target_names=["NO", "SI"], digits=3)
auc = roc_auc_score(y_test, y_proba)

print("\nExactitud (accuracy):", round(acc, 4))
print("ROC AUC:", round(auc, 4))
print("\nMatriz de confusión:\n", cm)
print("\nReporte de clasificación:\n", report)

# Curva ROC
fpr, tpr, thr = roc_curve(y_test, y_proba)
plt.figure(figsize=(6, 5))
plt.plot(fpr, tpr, label=f"ROC AUC = {auc:.3f}")
plt.plot([0, 1], [0, 1], linestyle="--")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("Curva ROC - Regresión Logística (L1)")
plt.legend()
plt.show()

# Importancia de variables (coeficientes) — requiere mapear nombres de features
# Obtenemos nombres después del OneHot + num
ohe = best_model.named_steps["preprocess"].named_transformers_["cat"]
num_feat_names = num_cols
cat_feat_names = ohe.get_feature_names_out(cat_cols).tolist()
feature_names = num_feat_names + cat_feat_names

coefs = best_model.named_steps["clf"].coef_.ravel()
coef_df = pd.DataFrame({"feature": feature_names, "coef": coefs})
coef_df["abs_coef"] = coef_df["coef"].abs()
coef_df = coef_df.sort_values("abs_coef", ascending=False)
display(coef_df.head(20))
